In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



# !mkdir -p drive

# !google-drive-ocamlfuse drive
# !cp /content/drive/My Drive/images.zip ./


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130963 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [3]:
!unzip drive/images.zip

Output hidden; open in https://colab.research.google.com to view.

In [0]:
import tensorflow as tf
from tensorflow.python.training.moving_averages import assign_moving_average
import numpy as np
import os
import cv2
import numpy as np
import random as rn
import tensorflow as tf
import threading
import time

global n_classes
n_classes = 50

In [5]:

_weights = {
        'wc1': tf.get_variable("wc1", [7, 7, 3, 96], initializer=tf.glorot_uniform_initializer()),
        'wc2': tf.get_variable('wc2',[5, 5, 96, 256], initializer=tf.glorot_uniform_initializer()),
        'wc3': tf.get_variable('wc3',[3, 3, 256, 384], initializer=tf.glorot_uniform_initializer()),
        'wc4': tf.get_variable('wc4',[3, 3, 384, 384], initializer=tf.glorot_uniform_initializer()),
        'wc5': tf.get_variable('wc5',[3, 3, 384, 256], initializer=tf.glorot_uniform_initializer()),
        'wd2': tf.get_variable('wd2',[4096, 4096], initializer=tf.glorot_uniform_initializer()),
        'out': tf.get_variable('out',[4096, n_classes], initializer=tf.glorot_uniform_initializer())
    }
_biases = {
        'bc1': tf.get_variable('bc1',[96], initializer=tf.glorot_uniform_initializer()),
        'bc2': tf.get_variable('bc2',[256], initializer=tf.glorot_uniform_initializer()),
        'bc3': tf.get_variable('bc3',[384], initializer=tf.glorot_uniform_initializer()),
        'bc4': tf.get_variable('bc4',[384], initializer=tf.glorot_uniform_initializer()),
        'bc5': tf.get_variable('bc5',[256], initializer=tf.glorot_uniform_initializer()),
        'bd2': tf.get_variable('db2',[4096], initializer=tf.glorot_uniform_initializer()),
        'out': tf.get_variable('bout',[n_classes], initializer=tf.glorot_uniform_initializer())
    }

W0710 09:46:26.846810 139941097412480 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
def activation(x,name="activation"):
    return tf.nn.relu(x, name=name)
    
def conv2d(name, l_input, w, b, s, p, scope):
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = activation(l_input+b)
    
    return l_input

def max_pool(name, l_input, k, s):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding='VALID', name=name)

def norm(l_input, lsize=4, name="lrn"):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)


In [0]:
   
def alex_net(_X, _dropout, batch_size):
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'], 3, 'VALID', 'conv1')
    pool1 = max_pool('pool1', conv1, k=3,s=2)
    conv2 = conv2d('conv2', pool1, _weights['wc2'], _biases['bc2'], 1, 'SAME', 'conv2')
    pool2 = max_pool('pool2', conv2, k=3,s=2)
    conv3 = conv2d('conv3', pool2, _weights['wc3'], _biases['bc3'], 1, 'SAME', 'conv3')
    conv4 = conv2d('conv4', conv3, _weights['wc4'], _biases['bc4'], 1, 'SAME', 'conv4')
    conv5 = conv2d('conv5', conv4, _weights['wc5'], _biases['bc5'], 1, 'SAME', 'conv5')
    pool5 = max_pool('pool2', conv5, k=3,s=2)
    # Find current size of conv5 to fit the requirement of FC1.
    sizes = pool5.get_shape().as_list()
    neurons =  sizes[1]*sizes[2]*sizes[3]
    dense1 = tf.reshape(pool5, [batch_size, neurons]) # Reshape conv5 output to fit dense layer input
    wei_den1 = tf.get_variable('wd3',[neurons, 4096], initializer=tf.glorot_uniform_initializer())
    b_den1 =  tf.get_variable('wd4',[4096], initializer=tf.glorot_uniform_initializer())
    
    dense1 = activation(tf.matmul(dense1, wei_den1) + b_den1, name='fc1') # Relu activation
    dd1=tf.nn.dropout(dense1, _dropout)
    dense2 = activation(tf.matmul(dd1, _weights['wd2']) + _biases['bd2'], name='fc2') # Relu activation
    out = tf.matmul(dense2, _weights['out']) + _biases['out'] # Relu activation

    return out

In [0]:
#==========================================================================
#=============Reading data in multithreading manner========================
#==========================================================================
def read_labeled_image_list(image_list_file, training_img_dir):
    """Reads a .txt file containing pathes and labeles
    Args:
       image_list_file: a .txt file with one /path/to/image per line
       label: optionally, if set label will be pasted after each line
    Returns:
       List with all filenames in file image_list_file
    """
    f = open(image_list_file, 'r')
    filenames = []
    labels = []

    for line in f:
        filename, label = line[:-1].split(' ')
        filename = training_img_dir+filename
        filenames.append(filename)
        labels.append(int(label))
        
    return filenames, labels
    
    
def read_images_from_disk(input_queue, size1=256):
    """Consumes a single filename and label as a ' '-delimited string.
    Args:
      filename_and_label_tensor: A scalar string tensor.
    Returns:
      Two tensors: the decoded image, and the string label.
    """
    label = input_queue[1]
    fn=input_queue[0]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    
    #example = tf.image.decode_png(file_contents, channels=3, name="dataset_image") # png fo rlfw
    example=tf.image.resize_images(example, [size1,size1])
    return example, label, fn
    
def setup_inputs(sess, filenames, training_img_dir, image_size=256, crop_size=224, isTest=False, batch_size=128):
    
    # Read each image file
    image_list, label_list = read_labeled_image_list(filenames, training_img_dir)

    images = tf.cast(image_list, tf.string)
    labels = tf.cast(label_list, tf.int64)
     # Makes an input queue
    if isTest is False:
        isShuffle = True
    else:
        isShuffle = False
        
    input_queue = tf.train.slice_input_producer([images, labels], shuffle=isShuffle)
    image, y,fn = read_images_from_disk(input_queue)

    channels = 3
    image.set_shape([None, None, channels])
        
    # Crop and other random augmentations
    if isTest is False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_saturation(image, .95, 1.05)
        image = tf.image.random_brightness(image, .05)
        image = tf.image.random_contrast(image, .95, 1.05)
        

    image = tf.random_crop(image, [crop_size, crop_size, 3])
    image = tf.cast(image, tf.float32)/255.0
    
    image, y,fn = tf.train.batch([image, y, fn], batch_size=batch_size, capacity=4,name='labels_and_images')

    tf.train.start_queue_runners(sess=sess)

    return image, y, fn, len(label_list)


In [0]:
# Training setting
batch_size = 128 
display_step = 80
dropout = 0.5# Dropout rate


In [10]:
keep_prob = tf.placeholder(tf.float32)          # Dropout rate to be fed
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
lr = 1e-3                                   # Learning rate start

# Setup the tensorflow...
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

print("Preparing the training & validation data...")
train_data, train_labels, filelist1, glen1 = setup_inputs(sess, "train.txt", "./", batch_size=batch_size)
val_data, val_labels, filelist2, tlen1 = setup_inputs(sess, "val.txt", "./", batch_size=batch_size)

max_iter = glen1*100

print("Preparing the training model with learning rate = %.5f..." % (lr))

with tf.variable_scope("alexnet", reuse=tf.AUTO_REUSE) as scope:
    pred = alex_net(train_data,keep_prob,batch_size)

with tf.name_scope('Loss_and_Accuracy'):
  cost = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=pred)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
  correct_prediction = tf.equal(tf.argmax(pred, 1), train_labels)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  top5=tf.reduce_mean(tf.cast(tf.nn.in_top_k(pred, train_labels, 5), tf.float32))
  
  tf.summary.scalar('Loss', cost)
  tf.summary.scalar('Training_Accuracy', accuracy)
  tf.summary.scalar('Top-5_accuracy', top5)


Preparing the training & validation data...


W0710 09:46:57.252592 139941097412480 deprecation.py:323] From <ipython-input-8-b45acdff6d74>:51: slice_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0710 09:46:57.264118 139941097412480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:374: range_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0710 09:46:57.285225 139941097412480 deprecation.py:3

Preparing the training model with learning rate = 0.00100...


W0710 09:46:57.985480 139941097412480 deprecation.py:506] From <ipython-input-7-5f240c5a2da9>:19: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0710 09:46:58.034357 139941097412480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess.run(init)
step = 0
writer = tf.summary.FileWriter("/tmp/log2", sess.graph)
summaries = tf.summary.merge_all()

print("We are going to train the ImageNet model based on AlexNet!!!")
while (step * batch_size) < max_iter:
    epoch1=np.floor((step*batch_size)/glen1)
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-3) & (epoch1 >2)):
        lr /= 10

    sess.run(optimizer,  feed_dict={keep_prob: dropout, learning_rate: lr})

    if (step % 15000==1) & (step>15000):
        save_path = saver.save(sess, "checkpoint/tf_alex_model_iter" + str(step) + ".ckpt")
        print("Model saved in file at iteration %d: %s" % (step*batch_size,save_path))

    if step % display_step == 1:
        # calculate the loss
        loss, acc, top5acc, summaries_string = sess.run([cost, accuracy, top5, summaries], feed_dict={keep_prob: 1.})
        print("Iter=%d/epoch=%d, Loss=%.6f, Training Accuracy=%.6f, Top-5 Accuracy=%.6f, lr=%f" % (step*batch_size, epoch1 ,loss, acc, top5acc, lr))
        writer.add_summary(summaries_string, step)


    step += 1
print("Optimization Finished!")
save_path = saver.save(sess, "checkpoint/tf_alex_model.ckpt")
print("Model saved in file: %s" % save_path)

We are going to train the ImageNet model based on AlexNet!!!
Iter=128/epoch=0, Loss=4.347581, Training Accuracy=0.031250, Top-5 Accuracy=0.093750, lr=0.001000
Iter=10368/epoch=0, Loss=3.781576, Training Accuracy=0.015625, Top-5 Accuracy=0.195312, lr=0.001000
Iter=20608/epoch=0, Loss=3.724192, Training Accuracy=0.039062, Top-5 Accuracy=0.171875, lr=0.001000
Iter=30848/epoch=0, Loss=3.618005, Training Accuracy=0.078125, Top-5 Accuracy=0.281250, lr=0.001000
Iter=41088/epoch=0, Loss=3.890863, Training Accuracy=0.046875, Top-5 Accuracy=0.203125, lr=0.001000
Iter=51328/epoch=0, Loss=3.567448, Training Accuracy=0.085938, Top-5 Accuracy=0.265625, lr=0.001000
Iter=61568/epoch=0, Loss=3.621604, Training Accuracy=0.046875, Top-5 Accuracy=0.257812, lr=0.001000
Iter=71808/epoch=1, Loss=3.319954, Training Accuracy=0.140625, Top-5 Accuracy=0.375000, lr=0.001000
Iter=82048/epoch=1, Loss=3.491551, Training Accuracy=0.093750, Top-5 Accuracy=0.312500, lr=0.001000
Iter=92288/epoch=1, Loss=3.277677, Traini

In [0]:
exit()